# 📰 News Summarization Model Demo

This notebook demonstrates a multi-strategy, company-trained news summarizer.  
It is designed for **prototyping and comparison**—you can use the dropdown for instant results, or enter your own article and (optional) guiding tags to try the model on anything.
---

> ⚡ **Quick Start**  
> - No manual model download or upload needed—the notebook will handle it for you.  
> - All dependencies are installed automatically.
> - If you just want to see outputs, scroll to the bottom for static examples.

---
# 📰 News Summarization Model Demo

---

## Table of Contents

- [How to use this notebook](#how-to-use-this-notebook)
- [Interactive Demo](#interactive-demo)
- [Automatic Model Download](#automatic-model-download)
- [Required Libraries](#required-libraries-automatically-installed)
- [Demo Results (Scroll Down)](#demo-results-scroll-down)
- [Example Summarization Outputs](#example-summarization-outputs)

---

This notebook demonstrates a multi-strategy, company-trained news summarizer.  
It is designed for **prototyping and comparison**—you can use the dropdown for instant results, or enter your own article and (optional) guiding tags to try the model on anything.


## How to use this notebook

1. **Run all cells** (first-time setup may take a few minutes).
2. Use the **dropdown** to view pre-loaded sample articles and summaries.
3. **Or:** Paste your own article and (optionally) guiding tags to test the model interactively.
   - *Guiding tags are **only needed** for Module 3 (Metadata-Guided) and Module 4 (Angled) summarizers. All other modules work without tags!*
4. View all summaries produced by different modules—compare and analyze!

---



> **Tip:**  
> If you don’t want to run the notebook, just scroll below to see real sample outputs from the model.


## Required Libraries (automatically installed):


In [1]:
!pip install -q transformers sentencepiece torch accelerate spacy yake rouge_score evaluate sentence-transformers rapidfuzz
!python -m spacy download en_core_web_sm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 8.0 MB/s eta 0:00:00
   ━

In [2]:
from google.colab import drive
import shutil
import zipfile
import os
# 1. Mount Google Drive
print("Step 1: Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive mounted successfully.")
except Exception as e:
    print(f"❌ Error mounting Google Drive: {e}")
    raise SystemExit("Stopping execution.")


Step 1: Mounting Google Drive...
Mounted at /content/drive
✅ Google Drive mounted successfully.


### Automatic Model Download

The next cell will automatically download the model zip file from a public Google Drive link and save it to your own Google Drive in the correct folder.  
**You do not need to manually download or upload anything!**

If you already have the model file in your Drive, you can skip this cell.


In [3]:
# Install gdown
!pip install -q gdown

import os, zipfile, shutil, glob
import gdown

# === Paths ===
drive_dir   = "/content/drive/My Drive/models"
os.makedirs(drive_dir, exist_ok=True)

zip_name    = "bart-english-news-summarizer.zip"
drive_zip   = os.path.join(drive_dir, zip_name)
local_zip   = os.path.join("/content", zip_name)
extract_dir = os.path.join(drive_dir, "bart-english-news-summarizer")
url         = "https://drive.google.com/uc?export=download&id=1JakrYwFLk00F1f9yofrLgJMlU3IQAxPN"

def clean_part_files():
    # remove any *.part files related to the zip in both places
    for pattern in [
        os.path.join("/content", zip_name + "*"),
        os.path.join(drive_dir, zip_name + "*"),
        os.path.join(drive_dir, "*.part"),
    ]:
        for p in glob.glob(pattern):
            if p.endswith(".part"):
                try:
                    os.remove(p)
                    print("Removed leftover part file:", p)
                except Exception as e:
                    print("Could not remove:", p, "-", e)

# Clean before
clean_part_files()

# === Download to local (fresh; avoid .part by not resuming) ===
if not os.path.exists(local_zip):
    print("Downloading model zip to local...")
    gdown.download(url=url, output=local_zip, quiet=False, resume=False)
else:
    print("Local zip already exists:", local_zip)

# Quick sanity check
if not os.path.exists(local_zip) or os.path.getsize(local_zip) < 100*1024*1024:
    raise RuntimeError("Local zip missing or too small; download likely failed.")

# === Copy to Drive if needed ===
if not os.path.exists(drive_zip):
    print("Copying zip into Drive…")
    with open(local_zip, "rb") as src, open(drive_zip, "wb") as dst:
        shutil.copyfileobj(src, dst, length=1024*1024)
    # verify copy
    if os.path.getsize(drive_zip) != os.path.getsize(local_zip):
        raise IOError("Drive copy size mismatch.")
else:
    print("Zip already present in Drive:", drive_zip)

# === Verify zip & extract ===
def extract_zip(zip_path, to_dir):
    try:
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.testzip()  # quick integrity check
        if not os.path.exists(to_dir) or not any(os.scandir(to_dir)):
            os.makedirs(to_dir, exist_ok=True)
            with zipfile.ZipFile(zip_path, "r") as zf:
                zf.extractall(to_dir)
            print("✅ Extracted to:", to_dir)
        else:
            print("Model folder already exists:", to_dir)
    except zipfile.BadZipFile:
        raise RuntimeError("Zip appears corrupted. Delete it and retry.")

extract_zip(drive_zip, extract_dir)

# Clean after
clean_part_files()

print("\nAll set. Use model path:")
print(extract_dir)


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1JakrYwFLk00F1f9yofrLgJMlU3IQAxPN
From (redirected): https://drive.google.com/uc?export=download&id=1JakrYwFLk00F1f9yofrLgJMlU3IQAxPN&confirm=t&uuid=9b16d486-9856-4d2b-825c-42a0341c8e11
To: /content/bart-english-news-summarizer.zip
100%|██████████| 1.51G/1.51G [00:16<00:00, 90.9MB/s]


Copying zip into Drive…
✅ Extracted to: /content/drive/My Drive/models/bart-english-news-summarizer

All set. Use model path:
/content/drive/My Drive/models/bart-english-news-summarizer


In [4]:

import os, zipfile, shutil, glob


drive_zip = "/content/drive/My Drive/models/bart-english-news-summarizer.zip"


target_dir = "/content/bart-english-news-summarizer"   # == "./bart-english-news-summarizer"
tmp_extract = "/content/_tmp_model_extract"



# Clean old target to avoid nested folders
if os.path.exists(target_dir):
    shutil.rmtree(target_dir)
os.makedirs(target_dir, exist_ok=True)


if os.path.exists(tmp_extract):
    shutil.rmtree(tmp_extract)
os.makedirs(tmp_extract, exist_ok=True)

if not os.path.exists(drive_zip):
    raise FileNotFoundError(f"Model zip not found at:\n{drive_zip}\nCheck the path.")

print("Extracting model zip...")
with zipfile.ZipFile(drive_zip, "r") as zf:
    zf.extractall(tmp_extract)


candidates = glob.glob(os.path.join(tmp_extract, "**", "config.json"), recursive=True)
if not candidates:
    raise FileNotFoundError(
        f"No config.json found inside {drive_zip}. "
        "The zip must contain a valid HF model folder."
    )


candidates.sort(key=lambda p: p.count(os.sep))
model_root = os.path.dirname(candidates[0])


shutil.copytree(model_root, target_dir, dirs_exist_ok=True)


cfg = os.path.join(target_dir, "config.json")
if not os.path.isfile(cfg):
    raise RuntimeError(f"config.json not found in {target_dir}.")
print("✅ Local model ready at:", target_dir)
print("Files:", os.listdir(target_dir)[:10])




Extracting model zip...
✅ Local model ready at: /content/bart-english-news-summarizer
Files: ['generation_config.json', 'config.json', 'model.safetensors', 'tokenizer_config.json', 'vocab.json', 'special_tokens_map.json', 'training_args.bin', 'tokenizer.json', 'merges.txt']


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer, util
from rapidfuzz import fuzz
import time
import spacy
from collections import Counter

# Control panel
MODEL_PATH = "bart-english-news-summarizer"
INPUT_TEXT = ""
GUIDING_TAGS = []
NUM_Candidates_FOR_RERANKING = 5



def sentence_chunker_adaptive(
    text, tokenizer, max_tokens=900, overlap_sentences=1, min_last_chunk_ratio=0.5, model_max_length=1024
):
    """
    Splits text into sentence-based chunks, each under max_tokens (by tokenizer).
    If the last chunk is < min_last_chunk_ratio * max_tokens, it merges with previous, as long as merged chunk doesn't exceed model_max_length.
    """
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    chunks = []
    current_chunk = []
    current_tokens = 0
    i = 0
    while i < len(sentences):
        sent = sentences[i]
        sent_tokens = len(tokenizer(sent, truncation=False, padding=False)["input_ids"])
        if current_tokens + sent_tokens > max_tokens:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
                # Overlap: keep last N sentences as context
                overlap = current_chunk[-overlap_sentences:] if overlap_sentences > 0 else []
                current_chunk = list(overlap)
                current_tokens = sum(len(tokenizer(s, truncation=False, padding=False)["input_ids"]) for s in overlap)
            else:
                # Single sentence is longer than chunk size
                chunks.append(sent)
                current_chunk = []
                current_tokens = 0
            continue
        current_chunk.append(sent)
        current_tokens += sent_tokens
        i += 1
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Adaptive merge: merge last chunk if it's too small, but not if merged > model_max_length
    if len(chunks) > 1:
        last_chunk_tokens = len(tokenizer(chunks[-1], truncation=False, padding=False)["input_ids"])
        merged = chunks[-2] + " " + chunks[-1]
        merged_tokens = len(tokenizer(merged, truncation=False, padding=False)["input_ids"])
        if last_chunk_tokens < min_last_chunk_ratio * max_tokens and merged_tokens <= model_max_length:
            chunks[-2] = merged
            chunks = chunks[:-1]

    # Final check: truncate any chunk over model_max_length
    for idx, chunk in enumerate(chunks):
        tokens = tokenizer(chunk, truncation=False, padding=False)["input_ids"]
        if len(tokens) > model_max_length:
            # Truncate to model_max_length tokens
            truncated = tokenizer.decode(tokens[:model_max_length], skip_special_tokens=True)
            chunks[idx] = truncated

    return chunks

def generate_single_summary(
    text, summarizer_pipeline, generation_params,
    max_chunk_tokens=900, overlap_sentences=1, min_last_chunk_ratio=0.5
):
    tokenizer = summarizer_pipeline.tokenizer
    model_max_length = summarizer_pipeline.model.config.max_position_embeddings
    token_ids = tokenizer(text, truncation=False, padding=False)["input_ids"]
    total_tokens = len(token_ids)

    if total_tokens <= model_max_length:
        result = summarizer_pipeline(text, **generation_params)
        return result[0]["summary_text"] if result else None

    # Chunk and summarize
    chunks = sentence_chunker_adaptive(
        text, tokenizer, max_tokens=max_chunk_tokens,
        overlap_sentences=overlap_sentences,
        min_last_chunk_ratio=min_last_chunk_ratio,
        model_max_length=model_max_length
    )

    chunk_summaries = []
    for idx, chunk in enumerate(chunks):
        chunk_params = generation_params.copy()
        chunk_params.setdefault("max_length", 140)
        chunk_params.setdefault("min_length", 40)
        summary_result = summarizer_pipeline(chunk, **chunk_params)
        if summary_result:
            chunk_summaries.append(summary_result[0]["summary_text"])

    return "\n".join(chunk_summaries)

###########################################################################


def deduplicate_candidates(
    candidates, threshold=0.90, model_name='all-MiniLM-L6-v2', verbose=False
):
    """
    Deduplicate (summary, label) pairs using SBERT similarity,
    keeping the longer summary in each similar pair.
    Prints debug info if verbose.
    """
    if not candidates:
        return []
    model = SentenceTransformer(model_name)
    summaries = [c[0] for c in candidates]
    embeddings = model.encode(summaries, convert_to_tensor=True)
    n = len(summaries)
    keep = [True] * n
    cut_pairs = []
    for i in range(n):
        if not keep[i]:
            continue
        for j in range(i + 1, n):
            if not keep[j]:
                continue
            sim = util.cos_sim(embeddings[i], embeddings[j]).item()
            if sim > threshold:
                len_i = len(summaries[i])
                len_j = len(summaries[j])
                # Keep the longer summary
                if len_j > len_i:
                    keep[i] = False
                    cut_pairs.append((i, j, sim))
                    break  # i has been cut, stop comparing further
                else:
                    keep[j] = False
                    cut_pairs.append((j, i, sim))
    if verbose and cut_pairs:
        print("\n--- Deduplication Debug Info (Keep Longer) ---")
        for cut_idx, kept_idx, sim in cut_pairs:
            print(f"\n[REMOVED] Summary {cut_idx+1} (from '{candidates[cut_idx][1]}')\n"
                  f"    {candidates[cut_idx][0]}\n"
                  f"    [Similarity: {sim*100:.2f}%]\n"
                  f"    --- too similar to ---\n"
                  f"[KEPT]    Summary {kept_idx+1} (from '{candidates[kept_idx][1]}')\n"
                  f"    {candidates[kept_idx][0]}\n")
        print("--- End Debug Info ---\n")
    deduped = [candidates[k] for k in range(n) if keep[k]]
    return deduped
def filter_hallucinated_entities(candidate_summaries, article_text, nlp=None, labels={"PERSON", "ORG", "GPE"}, fuzzy_threshold=90, verbose=False):
    """
    Filters out summaries that mention PERSON/ORG/GPE entities not present in the article, with fuzzy matching.
    Uses rapidfuzz for fuzzy string comparison (threshold in percent).
    """
    if nlp is None:
        nlp = spacy.load("en_core_web_sm")
    doc_article = nlp(article_text)
    article_ents = set(ent.text.lower() for ent in doc_article.ents if ent.label_ in labels)
    filtered = []
    for summary, label in candidate_summaries:
        doc_summary = nlp(summary)
        summary_ents = set(ent.text.lower() for ent in doc_summary.ents if ent.label_ in labels)
        hallucinated = set()
        for s_ent in summary_ents:
            match_found = False
            for a_ent in article_ents:
                # Fuzzy match
                if fuzz.ratio(s_ent, a_ent) >= fuzzy_threshold:
                    match_found = True
                    break
            if not match_found:
                hallucinated.add(s_ent)
        if hallucinated:
            if verbose:
                print(f"[FACT-CHECK WARNING] {label}")
                print(f"  Summary: {summary}")
                print(f"  Hallucinated entities: {hallucinated}\n")
            continue
        else:
            filtered.append((summary, label))
    return filtered
def module_1_baseline_generation(input_text, pipeline_beam, pipeline_sample):
    print("\n--> Running Module 1: Baseline Generation...")
    summaries = []
    beam_search_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": False,
        "num_beams": 4,
        "early_stopping": True,
        "repetition_penalty": 1.2,
    }
    summary_beam = generate_single_summary(
        input_text, pipeline_beam, beam_search_params
    )
    if summary_beam:
        summaries.append(summary_beam)
    sampling_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "top_k": 50,
        "top_p": 0.95,
        "temperature": 0.8,
    }
    summary_sample = generate_single_summary(
        input_text, pipeline_sample, sampling_params
    )
    if summary_sample:
        summaries.append(summary_sample)
    return summaries

def module_2_simple_prompters(input_text, pipeline_beam):
    print("\n--> Running Module 2: Simple Prompters...")
    summaries = []
    prompt_simple = f"Summarize this article in one concise sentence:\n\n{input_text}"
    simple_command_params = {
        "max_length": 60,
        "min_length": 10,
        "num_beams": 4,
        "early_stopping": True,
    }
    summary_simple = generate_single_summary(
        prompt_simple, pipeline_beam, simple_command_params
    )
    if summary_simple:
        summaries.append(summary_simple)
    prompt_cot = f"First, identify the key entities and the main event in the following article. Then, based on that analysis, provide a concise, abstractive summary.\n\nArticle:\n{input_text}"
    cot_params = {
        "max_length": 150,
        "min_length": 35,
        "num_beams": 4,
        "repetition_penalty": 1.2,
        "early_stopping": True,
    }
    summary_cot = generate_single_summary(prompt_cot, pipeline_beam, cot_params)
    if summary_cot:
        summaries.append(summary_cot)
    return summaries

def module_3_metadata_guided(input_text, guiding_tags, pipeline_beam, pipeline_sample):
    print("\n--> Running Module 3: Metadata-Guided Summarizers...")
    summaries = []
    if not guiding_tags:
        print("    - Skipping module: No GUIDING_TAGS provided.")
        return summaries
    tags_str = (
        ", ".join(guiding_tags) if isinstance(guiding_tags, list) else guiding_tags
    )
    prompt_tags = f"""
    Summarize the following news article. It is important that the summary focuses on the key topics of: {tags_str}.

    Article:
    {input_text}
    """
    beam_guided_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": False,
        "num_beams": 4,
        "early_stopping": True,
    }
    summary_beam_guided = generate_single_summary(
        prompt_tags, pipeline_beam, beam_guided_params
    )
    if summary_beam_guided:
        summaries.append(summary_beam_guided)
    sampling_guided_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "top_k": 50,
        "top_p": 0.95,
    }
    summary_sample_guided = generate_single_summary(
        prompt_tags, pipeline_sample, sampling_guided_params
    )
    if summary_sample_guided:
        summaries.append(summary_sample_guided)
    return summaries

def module_4_angled_summarizers(input_text, guiding_tags, pipeline_beam, pipeline_sample):
    print("\n--> Running Module 4: Angled Summarizers...")
    summaries = []
    if not guiding_tags:
        print("    - Skipping module: No GUIDING_TAGS provided.")
        return summaries
    general_beam_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": False,
        "num_beams": 4,
        "early_stopping": True,
    }
    summary_general_beam = generate_single_summary(
        input_text, pipeline_beam, general_beam_params
    )
    if summary_general_beam:
        summaries.append(summary_general_beam)
    general_sample_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "top_k": 50,
        "top_p": 0.95,
    }
    summary_general_sample = generate_single_summary(
        input_text, pipeline_sample, general_sample_params
    )
    if summary_general_sample:
        summaries.append(summary_general_sample)
    for tag in guiding_tags:
        print(f"    - Generating angled summary for tag: '{tag}'...")
        angle_prompt = f"Summarize the following article with a specific focus on '{tag}'.\n\nArticle:\n{input_text}"
        angle_params = {
            "max_length": 80,
            "min_length": 15,
            "do_sample": False,
            "num_beams": 4,
        }
        summary_angle = generate_single_summary(
            angle_prompt, pipeline_beam, angle_params
        )
        if summary_angle:
            summaries.append(summary_angle)
    return summaries

def module_5_beam_and_sampling_options(input_text, num_sampling_options, pipeline_beam, pipeline_sample):
    print("\n--> Running Module 5: Beam + Multiple Sampling Options...")
    summaries = []
    beam_search_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": False,
        "num_beams": 4,
        "early_stopping": True,
        "repetition_penalty": 1.2,
    }
    summary_beam = generate_single_summary(
        input_text, pipeline_beam, beam_search_params
    )
    if summary_beam:
        summaries.append(summary_beam)
    sampling_params_single = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "num_return_sequences": 1,
        "top_k": 50,
        "top_p": 0.95,
        "temperature": 0.9,
        "repetition_penalty": 1.2,
        "no_repeat_ngram_size": 3,
    }
    sample_summaries = []
    for i in range(num_sampling_options):
        summary_sample = generate_single_summary(
            input_text, pipeline_sample, sampling_params_single
        )
        if summary_sample:
            sample_summaries.append(summary_sample)
    if sample_summaries:
        summaries.extend(sample_summaries)
    return summaries

def module_6_auto_keyword_guided_yake(input_text, pipeline_beam, pipeline_sample):
    print("\n--> Running Module 6: Automated Keyword-Guided Summarizers (YAKE)...")
    summaries = []
    try:
        import yake
    except ImportError:
        return summaries
    kw_extractor = yake.KeywordExtractor(lan="en", n=3, dedupLim=0.9, top=5)
    keywords_yake = [kw for kw, score in kw_extractor.extract_keywords(input_text)]
    if not keywords_yake:
        return summaries
    tags_str_yake = ", ".join(keywords_yake)
    prompt_yake = f"""
    Summarize the following news article. It is important that the summary focuses on the primary topics of: {tags_str_yake}. Focus on the main event described.

    Article:
    {input_text}
    """
    yake_beam_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": False,
        "num_beams": 4,
    }
    summary_yake_beam = generate_single_summary(
        prompt_yake, pipeline_beam, yake_beam_params
    )
    if summary_yake_beam:
        summaries.append(summary_yake_beam)
    yake_sample_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "top_k": 50,
    }
    summary_yake_sample = generate_single_summary(
        prompt_yake, pipeline_sample, yake_sample_params
    )
    if summary_yake_sample:
        summaries.append(summary_yake_sample)
    return summaries

def module_7_keyword_reranker_simple(input_text, num_candidates, pipeline_sample, nlp_ner):
    print("\n--> Running Module 7: Keyword-Based Reranker (Simple Count)...")
    doc = nlp_ner(input_text)
    target_labels = ["ORG", "PERSON", "PRODUCT", "EVENT", "GPE"]
    entities = [ent.text.lower() for ent in doc.ents if ent.label_ in target_labels]
    keywords_for_reranking = [item for item, count in Counter(entities).most_common(5)]
    rerank_gen_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "num_return_sequences": 1,
    }
    internal_candidates = []
    for _ in range(num_candidates):
        summary = generate_single_summary(
            input_text, pipeline_sample, rerank_gen_params
        )
        if summary:
            internal_candidates.append(summary)
    if not internal_candidates:
        return []
    ranked_candidates = []
    for candidate in internal_candidates:
        score = sum(1 for kw in set(keywords_for_reranking) if kw in candidate.lower())
        ranked_candidates.append({"summary": candidate, "score": score})
    ranked_candidates.sort(key=lambda x: x["score"], reverse=True)
    best_summary = ranked_candidates[0]["summary"]
    return [best_summary]

def module_8_weighted_keyword_reranker(input_text, num_candidates, pipeline_sample, nlp_ner):
    print("\n--> Running Module 8: Weighted Keyword Reranker...")
    doc = nlp_ner(input_text)
    target_labels = ["ORG", "PERSON", "PRODUCT", "EVENT", "GPE"]
    all_entities = [ent.text.lower() for ent in doc.ents if ent.label_ in target_labels]
    keyword_weights = Counter(all_entities)
    rerank_gen_params = {
        "max_length": 150,
        "min_length": 30,
        "do_sample": True,
        "num_return_sequences": 1,
    }
    internal_candidates = []
    for _ in range(num_candidates):
        summary = generate_single_summary(
            input_text, pipeline_sample, rerank_gen_params
        )
        if summary:
            internal_candidates.append(summary)
    if not internal_candidates:
        return []
    ranked_candidates_weighted = []
    for candidate in internal_candidates:
        score = 0
        found_keywords = set()
        for keyword, weight in keyword_weights.items():
            if keyword in candidate.lower() and keyword not in found_keywords:
                score += weight
                found_keywords.add(keyword)
        ranked_candidates_weighted.append({"summary": candidate, "score": score})
    ranked_candidates_weighted.sort(key=lambda x: x["score"], reverse=True)
    best_summary_weighted = ranked_candidates_weighted[0]["summary"]
    return [best_summary_weighted]

def run_model_inference(INPUT_TEXT,GUIDING_TAGS):
    print("--- Starting Ensemble Generation Process ---")
    start_time_total = time.time()
    try:
        print("\nStep 1: Loading all required models...")
        device = 0 if torch.cuda.is_available() else -1
        tokenizer = AutoTokenizer.from_pretrained(f"./{MODEL_PATH}")
        model = AutoModelForSeq2SeqLM.from_pretrained(f"./{MODEL_PATH}")
        pipeline_beam = pipeline(
            "summarization", model=model, tokenizer=tokenizer, device=device
        )
        pipeline_sample = pipeline(
            "summarization", model=model, tokenizer=tokenizer, device=device
        )
        nlp_ner = spacy.load("en_core_web_sm")
        print("✅ All models loaded successfully.")

        print("\nStep 2: Generating candidate summaries from modules...")
        labeled_candidates = {}
        module_1_results = module_1_baseline_generation(
            INPUT_TEXT, pipeline_beam, pipeline_sample
        )
        for summary in module_1_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = "Module 1: Baseline Generation"
        module_2_results = module_2_simple_prompters(INPUT_TEXT, pipeline_beam)
        for summary in module_2_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = "Module 2: Simple Prompters"
        module_3_results = module_3_metadata_guided(
            INPUT_TEXT, GUIDING_TAGS, pipeline_beam, pipeline_sample
        )
        for summary in module_3_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = "Module 3: Metadata-Guided Summarizers"
        module_4_results = module_4_angled_summarizers(
            INPUT_TEXT, GUIDING_TAGS, pipeline_beam, pipeline_sample
        )
        for summary in module_4_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = "Module 4: Angled Summarizers"
        module_5_results = module_5_beam_and_sampling_options(
            INPUT_TEXT, NUM_Candidates_FOR_RERANKING, pipeline_beam, pipeline_sample
        )
        for summary in module_5_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = (
                    "Module 5: Beam + Multiple Sampling Options"
                )
        module_6_results = module_6_auto_keyword_guided_yake(
            INPUT_TEXT, pipeline_beam, pipeline_sample
        )
        for summary in module_6_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = (
                    "Module 6: Automated Keyword-Guided Summarizers (YAKE)"
                )
        module_7_results = module_7_keyword_reranker_simple(
            INPUT_TEXT, NUM_Candidates_FOR_RERANKING, pipeline_sample, nlp_ner
        )
        for summary in module_7_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = (
                    "Module 7: Keyword-Based Reranker (Simple Count)"
                )
        module_8_results = module_8_weighted_keyword_reranker(
            INPUT_TEXT, NUM_Candidates_FOR_RERANKING, pipeline_sample, nlp_ner
        )
        for summary in module_8_results:
            if summary not in labeled_candidates:
                labeled_candidates[summary] = "Module 8: Weighted Keyword Reranker"
        end_time_total = time.time()
        final_candidates_list = list(labeled_candidates.items())
        if not final_candidates_list:
            print(
                "No candidates were generated. The generation modules are currently empty."
            )
            return []
        else:
            print(
                f"Generated a total of {len(final_candidates_list)} unique candidates."
            )
            print(
                f"(Total generation time: {end_time_total - start_time_total:.2f} seconds)"
            )
            return final_candidates_list

    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
        return []
    finally:
        print("\n--- Script Finished ---")


Load Sample Data

In [6]:

samples = {
    "Article 1": {
        "text": """Turkish Foreign Minister Hakan Fidan said Wednesday that the latest talks between Russia and Ukraine in Istanbul marked another step toward ending the war, emphasizing that “every new achievement brings the sides one step closer to peace.”

In a statement following the third round of direct negotiations hosted by Türkiye, Fidan noted that the two parties agreed on the mutual exchange of at least 1,200 prisoners of war, along with new steps for the return of civilians, including children.

“We observed with satisfaction that the negotiations are moving in a more constructive and result-oriented direction,” he said, underlining Türkiye’s role as a facilitator.

Fidan said the delegations discussed concrete measures to advance technical consultations on a ceasefire and agreed to work toward forming joint working groups on political, humanitarian and military matters.

“Another brick has been laid in the construction of a joint will toward a solution,” he said. “Negotiations must be conducted with patience. The support and interest shown by the international community to the Istanbul meetings also reflect the global yearning for peace.”

The head of the Ukrainian delegation, Rustem Umerov, said Kyiv proposed organizing a presidential-level meeting by late August and thanked Türkiye for its facilitation.

Ukraine also signaled its readiness for a ceasefire without preconditions.

On the Russian side, negotiator Vladimir Medinsky confirmed the prisoner exchange and proposed short-term ceasefires to evacuate the wounded and retrieve fallen soldiers.

He said Moscow had reviewed a list of Ukrainian children for repatriation and was open to a fourth round of talks.""",
        "real_summary": """'We observed with satisfaction that negotiations are moving in a more constructive and result-oriented direction,' says Hakan Fidan, underlining Türkiye’s role as facilitator""",
    },
    "Article 2": {
        "text": """Turkish Foreign Minister Hakan Fidan said Friday he does not expect Israeli Prime Minister Benjamin Netanyahu to hesitate if conditions for annexing the occupied West Bank align with his vision.

He warned that the move would not serve Israel’s long-term interests.

“If Netanyahu believes the conditions are right, I frankly don’t think he would hesitate. His mindset is maximalist,” he told Turkish broadcaster NTV. “But this won’t benefit Israel over the next decades.”

Fidan criticized Israel’s military actions in the Gaza Strip as a “raging frenzy” that has turned into a global concern. He said most countries, apart from a few, are no longer standing by Israel in the face of its nearly two-year war in Gaza.

He pointed to the international shift, noting French President Emmanuel Macron’s support for recognizing Palestine. “Except for US politicians, we don’t see open support for the current Zionist mindset in Israel,” he said.

On ceasefire efforts, Fidan identified three sticking points: who will distribute aid in Gaza, the withdrawal of Israeli ground forces and assurances about the ceasefire’s continuation following a release of hostages by Hamas.

He emphasized that Israel is trying to make Gaza unlivable not only through destruction but also by starvation.

“Israeli intelligence officials are traveling country to country, trying to convince them to accept large numbers of Palestinian refugees. This is a project to make Gaza ‘without Palestinians,’” he said.

Regarding the Israeli parliament’s call for annexing the West Bank, approved July 23, Fidan said it directly undermines the two-state solution.

“You’re trying to destroy something that everyone recognizes based on the 1967 borders. If you don’t accept a negotiated two-state solution now, you open yourself to future risks,” he said.

Russia-Ukraine peace talks

Fidan commented on Türkiye’s role in peace negotiations between Russia and Ukraine, and said the latest trilateral meeting in Istanbul included mechanisms for humanitarian exchanges and broader discussions about a potential ceasefire summit.

He said both sides expressed a willingness to attend a proposed leaders’ summit in Türkiye hosted by Turkish President Recep Tayyip Erdogan, with US President Donald Trump, Russian President Vladimir Putin and Ukrainian President Volodymyr Zelenskyy in attendance.

“Putin and Zelenskyy have different preconditions for the ceasefire. Negotiators will have to work that out,” he said. “If these talks continue with the same intent, I believe we could see an interim solution in a few more rounds.”

On Türkiye’s Eurofighter Typhoon deal, Fidan said the process reached a phase where detailed technical work has been done and “we’ve reached the point of saying, ‘Let’s do this.’”

Fidan also announced progress on visa facilitation with the EU, highlighting a new regulation enabling multiple-entry Schengen visas for those who previously held one.

He said there is a positive momentum with the EU on broader issues such as visa liberalization, modernization of the customs union and reopening of European Investment Bank operations.

Addressing tensions between Iran and Israel, Fidan said he does not believe Iran would strike unless it was first attacked.

He noted both countries appear to be incorporating lessons from a 12-day war in June and might reassess their positions going forward.

On the upcoming Balkan Peace Platform meeting in Istanbul, Fidan said leaders from Bosnia and Herzegovina, Montenegro, Kosovo, North Macedonia, Serbia and Albania will join Türkiye to discuss shared concerns and leave “a good legacy for the future.”""",
        "real_summary": """Hakan Fidan calls Netanyahu’s mindset ‘maximalist,’ urges progress in Russia-Ukraine ceasefire talks""",
    },
    "Article 3": {
        "text": """Türkiye's fighter jet project, known as KAAN, has three prototypes; one will be used for land tests, and the other two will fly as of April 2026, the head of Turkish Aerospace Inc. (TAI) said on Friday.

Speaking at Anadolu's Technology Desk as part of the International Defense Industry Fair (IDEF) 2025, Mehmet Demiroglu said KAAN, which rolled out in 2023 and made its maiden flight in 2024, moved to an upper league.

The 17th edition of the six-day defense fair IDEF, starting on Tuesday, is being held simultaneously at the Istanbul Fair Center, Atatürk Airport, Wow Hotel, and Atakoy Marina.

The event, organized by KFA Fairs with support from Türkiye’s Defense Industries Secretariat (SSB) and the Turkish Armed Forces Foundation, has Anadolu as its global communication partner.

He said phase 1 for the production of KAAN is about to begin, adding: "We have completed all our work to launch phase 1.

"I don't want to say anything yet, but I can say that you can expect some surprises. All of this shows how far Türkiye has come and how important the KAAN platform is."

Four countries have built it, and there are consortia that are trying to build it; now, a consortium that began with Türkiye and was announced at the Paris Airshow in 2018 is said to be planning its first flight around 2035-2040, he noted.

He said this demonstrates Türkiye's speed, belief, and capabilities. "We do not view KAAN as merely an aircraft project. KAAN is a journey toward reaching the pinnacle of a nation's defense industry, not just as TAI alone, but as a collective effort."

The aircraft, for which the project began in 2016, was rolled out in March 2023 and made its maiden flight in February 2024.

In May 2023, the name of the aircraft was announced by President Recep Tayyip Erdogan as KAAN.


- Hurjet exports

Touching on the jet trainer Hurjet's export deal to Spain, he said there will be a supply agreement in September or October this year with Spain.

In May, Spain signed a memorandum of understanding (MoU) with Türkiye to import Hurjet.

Underlining that Hürjet is the best in its class, he said Türkiye is in talks with countries that want to replace their training aircraft fleets in the 2030s.

He said: "There are countries among them that we would never have expected.

"In fact, I can say that we missed out on the first phase in Malaysia, but we will focus on the second phase. We know it will be difficult, but we will work to secure the second phase. Indonesia has serious potential. We believe there is serious potential in the Gulf countries."

Hurjet is a platform that is a candidate for training aircraft not only for Spain and Türkiye but also for other NATO countries, he said, adding: "This is also a platform that is a candidate for both training and attack aircraft in relatively smaller air forces."

"It has a munitions capacity of approximately three tons. Considering that F-16s have a payload capacity of around six tons, when we look at cost-effectiveness and operational costs, the Hurjet will be cheaper than the F-16.

"Therefore, we believe the Hurjet has a bright future," he added.""",
        "real_summary": """Turkish Aerospace completes all our work to launch phase 1 for production of KAAN, general manager says""",
    },
    "Article 4": {
        "text": """The Freedom Flotilla Coalition (FFC) announced Thursday that it has lost contact with the "Handala," a ship carrying humanitarian aid to Gaza as part of the group's mission to break Israel's blockade.

“All communications with the Handala's crew have been jammed,” the coalition said in a statement posted on its official Telegram channel.

“We lost all contact with our crew, and there are multiple drones near the vessel,” it said, “which means that they could have been intercepted or attacked.”

The coalition urged its supporters “to pressure for the safety of the crew,” calling on people to contact their representatives and local media to "pressure Israel to let 'Handala' go and guarantee a safe passage to Gaza."

No other details were available regarding the ship’s precise location, the status of its crew, or confirmation of an Israeli intervention.

The incident follows previous confrontations.

On May 2, the MV Conscience, a ship belonging to the Freedom Flotilla Coalition and transporting aid to Gaza, was attacked by drones while in international waters near Malta, causing a fire and structural damage.

On June 9, Israel intercepted another aid ship, the Madlene, in international waters off the coast of Gaza, detaining and later deporting its 12 international activists, including Swedish activist Greta Thunberg and French Member of European Parliament Rima Hassan, under the condition they not attempt to return.

Israel has killed more than 59,500 Palestinians, most of them women and children, in the Gaza Strip since October 2023. The military campaign has devastated the enclave, collapsed the health system and led to severe food shortages.

Last November, the International Criminal Court issued arrest warrants for Israeli Prime Minister Benjamin Netanyahu and his former Defense Minister Yoav Gallant for war crimes and crimes against humanity in Gaza.

Israel also faces a genocide case at the International Court of Justice for its war on the enclave.


""",
        "real_summary": """Drones seen near vessel, coalition says, suggesting potential attack or interception""",
    },
    "Article 5": {
        "text": """Officials from Azerbaijani defense firms participating in the International Defense Industry Fair (IDEF) 2025 in Istanbul praised Türkiye’s defense sector and its strong relations with Azerbaijani companies, highlighting the distinct approach Turkish suppliers take toward their neighbor.

Speaking to Anadolu at Azerbaijan's national pavilion at IDEF 2025, ITX-Motors Group Defense Director Zafer Cetinkaya stated that no matter what product they purchase, they ultimately find value for money in Türkiye.

"Of course, due to the close proximity between the two countries, Turkish suppliers approach Azerbaijan differently and provide greater convenience," he said.

Cetinkaya also stressed that Türkiye has come a long way in both the automotive and defense industries since the 1970s.

He noted that ITX has been working on armored vehicles for about two years, with one vehicle undergoing government testing in Azerbaijan for a month.

"And we've finally successfully completed the tests ourselves. We're ready to put it into service with the armed forces by the end of the year," he added.

Cetinkaya also remarked that this year’s IDEF was the most active in the past three or four years and that participation was quite high.


"The presence of Turkish companies like Baykar motivates us"

Synapline CEO Bekir Guliyev shared that his company was exhibiting a newly developed unmanned aerial vehicle (UAV). "This UAV is autonomous, and its unique feature is that it can fly in GPS-free environments. We have a lot of support from Türkiye. The presence of companies like Baykar motivates us."

It was Synapline’s first time participating in IDEF, which Guliyev said featured a wide array of Turkish companies across multiple sectors.


"Brother next to brother — this gives us both strength and inspiration"

Bahruz Baghirov, Engineering Representative for RD Smart, said they anticipate collaborating with Turkish firms in the near future. "Brother next to brother — this gives us both strength and inspiration."

He noted that RD Smart is participating in the fair with five drones, including three kamikaze models.

Mehdi Mehdiyev, USH Group Chief Commercial Officer, praised the IDEF's high level organization and noted this year’s strong turnout.

"By participating as a brother country, we are both greeting our brother country and also want to showcase our own products here," Mehdiyev said.

"The Turan-1 we brought here is a 7.62x39 caliber platform, designed entirely by us in collaboration with the special operations unit. It was developed based on their requests and has presented a very good solution in the region, both in terms of exclusivity, sustainability, and reliability. We also have products for the 5.56 platform," he added.

Azerbaijani firms showcased their leading defense, aviation, and aerospace technologies from their dedicated national pavilion at IDEF 2025.

Held across multiple venues — including the Istanbul Fair Center, Ataturk Airport, WOW Hotel, and Atakoy Marina — the six-day fair opened Tuesday.

The event is organized by KFA Fairs with support from Türkiye’s Defense Industries Secretariat and the Turkish Armed Forces Foundation. Anadolu is the event’s global communication partner.

This year’s fair hosts ministers, chiefs of staff, commanders, and top representatives from 103 countries, with 44 countries operating their own pavilions.

Over 900 domestic and 400 international defense companies are taking part.""",
        "real_summary": """Turkish suppliers offer Azerbaijan unique advantages due to close ties, says ITX-Motors Group Defense Director Zafer Cetinkaya """,
    },
    "Article 6": {
        "text": """Turkish astronaut Tuva Cihangir Atasever spoke at an event in London on Friday, highlighting Türkiye’s growing space program and the scientific effect of its first manned missions.

The gathering at the Turkish Embassy was attended by Turkish Ambassador to London Osman Koray Ertas, Atasever, and numerous guests.

“Not only space itself, but the scientific research conducted there holds the potential to change human history and the course of time,” Ertas said, adding that Türkiye has accelerated efforts and completed two manned missions.

“There is a strategic mindset behind the investments of major countries and companies in space. We must continue our efforts as well. We can see that these efforts will bring dramatic changes to many areas of life,” he added.

Ertas noted that Türkiye’s participation in the global race is crucial and described Atasever as one of its key contributors.

Atasever said 20 different scientific experiments were conducted during the two missions. “We took our first steps in scientific work through manned space missions,” he said.

Quoting fellow astronaut Alper Gezeravci, he added: “We all take hundreds or thousands of steps every day. Because they are so routine, we don’t care about them. But every parent treasures their child’s first step. We took our first step with scientific work.”

He said the experiments were valuable in building technical and operational expertise in Türkiye.

Atasever noted that the goal is to establish a “Microgravity Research Program” in Türkiye and efforts toward it have been ongoing since last July.

He also delivered a presentation about space missions, preparation stages and the future of scientific research in microgravity.

Türkiye completed its first manned spaceflight to the International Space Station in January 2024, followed by a suborbital flight in June.""",
        "real_summary": """Tuva Cihangir Atasever says Türkiye carried out 20 experiments during its first human spaceflights""",
    },
    "Article 7": {
        "text": """President Donald Trump signed executive orders aimed at changing K-12 education (kindergarten through high school) and college funding in line with conservative values.

Among the most controversial steps signed late Wednesday are proposals to withhold federal funding from universities that fail to report financial ties to foreign sources.

He also pushed to reduce diversity programs in higher education by pressuring accrediting organizations that oversee schools receiving federal aid, according to the Politico news outlet.

Trump called for new federal rules on school discipline that oppose previous Democratic policies. He signed orders to introduce AI training for students, update job programs and start a new White House initiative for historically Black colleges and universities.

Overall, the actions aim to tighten financial pressure on universities and expand Trump’s influence on the education system.

“Today’s Executive Orders pave the way for critical innovations — inviting more competition in the higher education accreditation system, ensuring transparency in college finances, supporting new technologies in the classroom, and more,” Education Secretary Linda McMahon wrote on X.

Colleges are already legally required to report foreign gifts or contracts of $250,000 or more, but conservatives have criticized the Biden administration for how those rules have been enforced.

Trump’s new order allows federal grants to be cut from universities that do not meet updated foreign funding disclosure rules, following recent freezes on aid to Columbia and Harvard for their handling of pro-Palestinian protests.

It also directs McMahon to undo Biden-era policies that allowed secrecy around foreign funds, and to require schools to reveal the source and purpose of such funding.

The order also pushes for changes in the college accreditation system, targeting groups that promote diversity efforts, including major accrediting bodies for law and medical schools.

“Federal recognition will not be provided to accreditors engaging in unlawful discrimination in violation of Federal law,” the order stated.

Accreditors are non-governmental bodies that assess colleges to confirm they meet required standards before qualifying for federal funding. During his 2024 campaign, Trump referred to accreditors as the “secret weapon” to weed liberal influence out of higher education.

The order follows Trump’s January push to stop “indoctrination” in schools, investigate campus protests and expand school choice. He recently moved to start closing the Education Department.

Accreditors have rejected Trump’s portrayal of their role but expressed openness to collaboration with the administration.

“We firmly reject President Trump’s mischaracterization,” said Heather Perfetti, president of the Middle States Commission on Higher Education, while emphasizing their willingness to work with the education secretary to advance goals of “quality, innovation, integrity, and accountability.”

Trump ordered a report on school discipline and demanded model policies "rooted in American values," criticizing concerns from the Biden administration about racial disparities in discipline.

Randi Weingarten, president of the American Federation of Teachers, responded.

“Despite their mantra of local control, the Trump Administration really does want to be in the business of education after all,” she said.""",
        "real_summary": """White House allegedly targets advancing conservative agenda in education""",
    },
    "Article 8": {
        "text": """Türkiye's missile producer Roketsan signed new foreign and domestic strategic cooperation agreements at the International Defense Industry Fair (IDEF) 2025 in Istanbul.

The 17th edition of the six-day defense fair IDEF, starting on Tuesday, is being held simultaneously at the Istanbul Fair Center, Ataturk Airport, Wow Hotel, and Atakoy Marina.

The event, organized by KFA Fairs with support from Türkiye's Defense Industries Secretariat (SSB) and the Turkish Armed Forces Foundation, has Anadolu as its global communication partner.

Roketsan attracted great interest at the fair with its innovative products and field-proven systems, becoming one of the most notable companies at the event.

The company finalized critical agreements covering product deliveries and new projects during signing ceremonies held on the first two days of the fair.


Artillery systems cooperation with Azerbaijan

Roketsan took its strategic partnership with Azerbaijan in the defense sector a step further by signing a new cooperation protocol in the field of artillery systems.

The “Artillery Systems Cooperation Protocol,” which came into effect on July 22, was signed by Agil Gurbanov, Deputy Minister of Defense of the Republic of Azerbaijan, and Faruk Yigit, Chairman of the Board of Directors of Roketsan.


Historic Agreement on Gokbora Project

The development contract for the Gokbora Beyond Visual Range Air-to-Air Missile project, which was introduced at IDEF 2025, was also signed on Wednesday between the Secretariat of Defense Industries (SSB) and Roketsan.

The contract was signed by SSB President Haluk Gorgun and Roketsan General Manager Murat Ikinci.

The Gokbora, which will be used with both manned and unmanned combat aircraft, is planned to become the new striking power of the air force with its range exceeding 100 nautical miles and superior guidance capability.

Additionally, a contract was signed between Roketsan and another Turkish defense firm Aselsan for the development of RF seeker heads and missile-mounted data link subsystems under the project.


Strategic agreements with SSB

Two more important agreements were also signed between Roketsan and the SSB.

The two institutions signed the Mixed Rocket/Missile System Project Agreement and the TRLG-122 Missile System Supply Cooperation Protocol.

These agreements aim to develop and procure critical systems that will advance Türkiye's defense capabilities.


Fuel rod agreement with MKE

Roketsan and a Turkish defense firm MKE also signed an agreement for the supply of fuel rods to be used in various projects.

The agreement was signed by Roketsan General Manager Murat Ikinci and MKE General Manager Ilhami Keles.


Deal with Teknopark Istanbul

Roketsan also signed a new protocol with Teknopark Istanbul to support new ventures in the defense and technology sectors.

The “Incubation Center-Focused Corporate Cooperation Protocol” was signed by Roketsan General Manager Murat Ikinci and Teknopark Istanbul General Manager Abdurrahman Akyol.

This cooperation aims to commercialize innovative ideas, support startups, and contribute to the defense ecosystem.


Localization efforts

On Thursday, Roketsan inked “Memorandum of Understanding on Localization Efforts” with companies it has collaborated with so far; Armsto Konnektor, Asil Celik, Sensorsan Sensor Technologies, Iveo Electronic Defense Systems, Korel Elektronik, and Pavezyum Kimya.

Roketsan's Rise for Tomorrow platform, launched last year, also plays a significant role in the localization initiative.

The platform, which was launched to increase the effectiveness of nationalization efforts, strengthen supplier relationships and the ecosystem with potential suppliers, is positioned as a sharing platform established to minimize supply risks and dependence on foreign countries.

Halid Bulut said that it is very important for Roketsan to continue its localization efforts without slowing down in order to reduce dependence on external sources in the supply process and become more independent.""",
        "real_summary": """Roketsan attracts great interest at defense fair with its innovative products """,
    },
    "Article 9": {
        "text": """With record-breaking temperatures, severe drought and a surge in wildfires, the summer of 2025 is shaping up as one of the most extreme in UK history.

After an exceptional spring and a sweltering June, this summer’s first half has underscored a shifting pattern: hotter days, longer dry spells and recurring heat waves are no longer rare outliers but part of the landscape.

“Spring 2025 was exceptional, going down as the UK’s warmest and sunniest on record,” the UK Met Office, the national meteorological service, said.

That heat carried into June, which became the warmest ever in England, and the second warmest for the UK overall in records dating back to 1884.

As of 15 July, the UK’s mean summer temperature was running 1.14C (2.1F) above the long-term average — with England even higher at 1.58C (2.8F). Maximums in England were 2.14C (3.85F) above average.

While the country hasn’t reached its all-time high of 40.3C (104.5F) set in July 2022, the heat has still been punishing. On 1 July, Faversham in Kent recorded the year’s top temperature at 35.8C (96.4F), while London hit 34.7C (94.5F).

“It’s hard to say this summer has been typical so far when we look at the statistics,” said Emily Carlisle, a Met Office climate scientist, in a statement. “Warmth has continued into July, with three heat waves so far this summer and temperatures rising again later this week. There is, of course, still a month-and-a-half of meteorological summer to go, so plenty of time for things to change.”

The dry conditions are just as extreme. England saw its driest January-to-June stretch since 1976, triggering drought alerts in multiple regions.

Wildfire risk has soared. The London Fire Brigade has responded to more than 30 wildfires this year, putting 2025 on track to be the busiest fire season since 2022.

National Resilience data revealed a sharp rise in wildfire incidents across England and Wales — 564 between January and mid-June, a 717% increase from the same period in 2024, and more than double 2022’s count at that time, which ended as the UK’s worst wildfire year on record.


A clear pattern

The UK’s changing climate is charted in the latest State of the UK Climate report, published in the International Journal of Climatology by the Royal Meteorological Society.

The data show steady warming since the 1980s, with extreme temperature days becoming more frequent. The hottest summer days and coldest winter nights are warming at twice the pace of average seasonal temperatures in some regions.

The Met Office said 2024 was the fourth warmest year since 1884 – and 2025 is already on track to surpass it in several categories.

A recent Met Office study found there’s now a 50% chance of the UK hitting 40C (104F) again within the next 12 years — a sharp jump in probability.

With three heat waves already recorded and another warm surge on the way, summer 2025 could become a defining chapter in Britain’s climate history.""",
        "real_summary": """‘It’s hard to say this summer has been typical so far when we look at statistics,’ says national meteorological service scientist""",
    },
    "Article 10": {
        "text": """GENEVA

Türkiye has the potential to become a regional leader by demonstrating how bold climate action can drive economic growth, strengthen energy security, and attract global investment, according to the UN’s top climate official.

Speaking to Anadolu about his visit to Türkiye this week, Simon Stiell, executive secretary of the UN Framework Convention on Climate Change (UNFCCC), praised the country’s clean energy progress, stressing that it is well-positioned for further steps through a new national climate plan.

“Türkiye can lead by example, especially within the region, by showing how climate targets go hand-in-hand with economic growth,” Stiell said.

“Türkiye has unique advantages – huge capacity for wind and solar, vibrant small and medium enterprises, and a young population. These advantages, combined with stronger targets, are a recipe for success.”

Stiell urged Türkiye to build on its current clean energy momentum and submit an enhanced Nationally Determined Contribution (NDC) this year.

“In 2024, Türkiye was in the top 10 countries with the largest absolute solar capacity additions,” he said.

“A more ambitious NDC, a new national climate plan and targets, would send a strong signal to global investors that Türkiye is committed to accelerating the transformation in its energy and power sectors – going further and faster to meet rising demand.”

He noted that stronger climate commitments could yield wide-ranging benefits: “It can save billions of dollars through reduced fossil fuel imports, creating hundreds of thousands of jobs, all while reducing its current account deficit, which will have very positive implications for the economy.

“In 2023, Türkiye attracted $1.4 billion for non-hydro renewables. Solar, wind, and hydrogen are all poised to take off.”

According to Stiell, the submission of a strengthened NDC is “a real opportunity” to boost energy independence and create lasting economic growth.

Transition leadership and a just future

Stiell highlighted the global shift towards clean energy and said countries that invest early are already seeing significant returns.

“Germany’s energy transition created a large clean energy workforce. Morocco’s solar investments positioned it as a regional energy leader,” he said. “India wants to become a solar superpower. Nigeria sees huge opportunities across society to reduce dependence on diesel generators.”

He also emphasized the central role of the private sector.

“The private sector’s role is particularly important, and the returns for the private sector stand to be significant, across every major economy,” he said.

A just transition must also remain a priority: “A just transition plan, including retraining and investment in affected communities, is also essential. Türkiye can lead the world in ensuring no worker or community associated with the fossil fuel industry is left behind.”

On whether tougher climate targets pose an economic burden, Stiell pointed to the importance of international cooperation and financial mechanisms.

“International climate finance, technology partnerships, and regional cooperation are key. Access to EU climate-linked funding can help reduce upfront transition costs. Clear policy frameworks and stronger NDCs can also unlock investment from the private sector by improving market confidence and reducing regulatory risk,” he said.

He also pointed to the role of Türkiye’s Emissions Trading System (ETS) in supporting competitiveness.

In other countries, similar carbon pricing mechanisms have generated revenues that are reinvested to fund clean energy efforts, he noted.

Business readiness and climate resilience

During his two-day visit to Türkiye that wrapped up Wednesday, Stiell met with Turkish civil society and business leaders, noting strong enthusiasm and readiness to contribute to the country’s green transition.

“On my trip I’ve heard how clear policies and climate ambition can support national growth, drive innovation, and create export opportunities. The private sector in this country is dynamic. Early action gives companies a competitive edge in global markets,” he said.

“And it isn’t just about existing energy – Turkish companies have all the tools to become a leader in new forms of green technology and manufacturing.”

Stiell also stressed the importance of policymaking that includes civil society, especially youth.

“Their representation in climate action and policymaking will ensure NDCs are well-aligned with national and local needs, and ensure that the most vulnerable segments of society, including women, youth, and poorer people, participate and benefit from the transition,” he said.

Stronger climate targets, he added, must also focus on adaptation and resilience.

“Stronger NDCs help direct investment not only into emissions reduction but also into building resilience. Türkiye is already facing serious climate risks, including wildfires and water scarcity,” he said.

“A well-designed national climate plan can support early warning systems, disaster risk reduction, drought and wildfire management, and climate-resilient infrastructure, which are essential for safeguarding hard-won economic gains and achieving long-term development targets.”

Stiell welcomed Türkiye’s progress on a national adaptation strategy and said sharing lessons with other nations will be key.

“A new national strategy on adaptation, launched last year, can help build momentum, and the submission of a National Adaptation Plan to the UN can help other countries learn lessons about what works.”""",
        "real_summary": """‘Türkiye can lead by example, especially within the region, by showing how climate targets go hand-in-hand with economic growth,’ says Simon Stiell, head of the UN Framework Convention on Climate Change""",
    },
}

In [7]:
import ipywidgets as widgets
from IPython.display import display, Markdown


# --- Widgets ---
dropdown = widgets.Dropdown(
    options=list(samples.keys()), description='Select Article:'
)
custom_text = widgets.Textarea(
    value='', placeholder='Paste your own article here...',
    description='Custom Article:', layout=widgets.Layout(width='99%', height='100px')
)
tags_text = widgets.Text(
    value='', placeholder='Comma-separated, e.g. Russia, Ukraine, economy',
    description='Guiding Tags:', layout=widgets.Layout(width='60%')
)
submit_button = widgets.Button(description="Run On Custom Input", button_style='primary')

# --- Instructions (Markdown) ---
display(Markdown("""
> **Prototype Info:**
> - For quick testing, use the dropdown to see pre-filled sample results.
> - For custom input, paste your own article and (optionally) comma-separated guiding tags below, then click 'Run On Custom Input'.
> - **Guiding tags are **_only needed_** for Module 3 (Metadata-Guided) and Module 4 (Angled) summarizers.
> - All other modules work without tags!
"""))

# --- Output area ---
output = widgets.Output()

def display_results(article_text, guiding_tags, real_summary=None):
    output.clear_output()
    with output:
        display(Markdown(f"### Article Text:\n\n{article_text}"))
        if real_summary:
            display(Markdown(f"### Real Summary:\n\n{real_summary}"))
        display(Markdown('---'))

        summaries = run_model_inference(article_text, guiding_tags)
        # summaries = filter_hallucinated_entities(summaries, article_text, verbose=False)
        summaries = deduplicate_candidates(summaries, threshold=0.91, verbose=False)

        if not summaries:
            display(Markdown("**No summaries were generated.**"))
        else:
            for i, (summary, module_name) in enumerate(summaries, 1):
                display(Markdown(f"### Generated Summary {i} (by {module_name}):\n\n{summary}"))

def on_article_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        article = samples[change['new']]

        display_results(article['text'], [], article.get('real_summary'))

def on_submit_button_click(b):
    article_text = custom_text.value.strip()
    tags_raw = tags_text.value.strip()
    guiding_tags = [t.strip() for t in tags_raw.split(',')] if tags_raw else []
    display_results(article_text, guiding_tags)

# --- Widget connections ---
dropdown.observe(on_article_change)
submit_button.on_click(on_submit_button_click)

# --- UI display ---
display(dropdown)
display(Markdown('<hr style="border:2px solid #0077B6">'))
display(Markdown("<b>Or use your own article:</b>"))
display(custom_text)
display(tags_text)
display(submit_button)
display(output)

# Show default article on load
on_article_change({'type':'change','name':'value','new':dropdown.value})


> **Prototype Info:**
> - For quick testing, use the dropdown to see pre-filled sample results.
> - For custom input, paste your own article and (optionally) comma-separated guiding tags below, then click 'Run On Custom Input'.
> - **Guiding tags are **_only needed_** for Module 3 (Metadata-Guided) and Module 4 (Angled) summarizers.
> - All other modules work without tags!


Dropdown(description='Select Article:', options=('Article 1', 'Article 2', 'Article 3', 'Article 4', 'Article …

<hr style="border:2px solid #0077B6">

<b>Or use your own article:</b>

Textarea(value='', description='Custom Article:', layout=Layout(height='100px', width='99%'), placeholder='Pas…

Text(value='', description='Guiding Tags:', layout=Layout(width='60%'), placeholder='Comma-separated, e.g. Rus…

Button(button_style='primary', description='Run On Custom Input', style=ButtonStyle())

Output()

> ## 📢 **Demo Results (Scroll Down)**
> ---
> The following are precomputed, static outputs from the model.
> You can compare these to live model outputs.


---

# 📋 Example Summarization Outputs

*Outputs are produced without tag input, no preprocessing*  
*1 Politics, 2 World, 1 Turkiye, 1 Economy, 2 Science-Technology, 1 Education, 2 Environment*

---

### Article: (Politics)
[Read the article](https://www.aa.com.tr/en/politics/turkish-foreign-minister-says-latest-russia-ukraine-talks-mark-another-step-toward-peace/3640334)

**Real Summary:**  
We observed with satisfaction that negotiations are moving in a more constructive and result-oriented direction,' says Hakan Fidan, underlining Türkiye’s role as facilitator

**Generated Summary:**  
*Generated by: Module 5: Beam + Multiple Sampling Options*  
‘We observed with satisfaction that the negotiations are moving in a more constructive and result-oriented direction,’ says Hakan Fidan after 3rd round of direct negotiations hosted by Türkiye

---

### Article: (World)
[Read the article](https://www.aa.com.tr/en/middle-east/turkish-foreign-minister-says-west-bank-annexation-would-backfire-on-israel/3642482)

**Real Summary:**  
Hakan Fidan calls Netanyahu’s mindset ‘maximalist,’ urges progress in Russia-Ukraine ceasefire talks

**Generated Summary:**  
*Generated by: Module 1: Baseline Generation*  
If Netanyahu believes the conditions are right, I frankly don’t think he would hesitate. His mindset is maximalist,’ says Hakan Fidan

---

### Article: (Turkiye)
[Read the article](https://www.aa.com.tr/en/turkiye/turkish-fighter-jet-kaan-will-fly-in-april-2026/3642389)

**Real Summary:**  
Turkish Aerospace completes all our work to launch phase 1 for production of KAAN, general manager says

**Generated Summary:**  
*Generated by: Module 8: Weighted Keyword Reranker*  
Phase 1 of production of KAAN aircraft about to begin, says head of Turkish Aerospace Inc. (TAI) Mehmet Demiroglu

---

### Article: (World)
[Read the article](https://www.aa.com.tr/en/middle-east/freedom-flotilla-coalition-loses-contact-with-aid-ship-handala-en-route-to-gaza/3641402)

**Real Summary:**  
Drones seen near vessel, coalition says, suggesting potential attack or interception

**Generated Summary:**  
*Generated by: Module 5: Beam + Multiple Sampling Options*  
Communications with 'Handala's crew have been jammed, multiple drones near vessel, which means they could have been intercepted or attacked,' says Freedom Flotilla Coalition

---

### Article: (Economy)
[Read the article](https://www.aa.com.tr/en/economy/azerbaijani-firms-showcase-strength-of-defense-partnership-with-turkiye/3641762)

**Real Summary:**  
Turkish suppliers offer Azerbaijan unique advantages due to close ties, says ITX-Motors Group Defense Director Zafer Cetinkaya

**Generated Summary:**  
*Generated by: Module 1: Baseline Generation*  
Turkish suppliers approach Azerbaijan differently and provide greater convenience, says ITX-Motors Group Defense Director Zafer Cetinkaya at Azerbaijan's national pavilion at IDEF 2025

---

### Article: (Science-Technology)
[Read the article](https://www.aa.com.tr/en/turkiye/turkish-astronaut-atasever-our-first-scientific-steps-began-with-space-missions-/3642480)

**Real Summary:**  
Tuva Cihangir Atasever says Türkiye carried out 20 experiments during its first human spaceflights

**Generated Summary:**  
*Generated by: Module 6: Automated Keyword-Guided Summarizers (YAKE)*  
Turkish astronaut Tuva Cihangir Atasever highlights Türkiye’s growing space program and scientific effect of its first manned missions

---

### Article: (Education)
[Read the article](https://www.aa.com.tr/en/americas/trump-unveils-executive-orders-to-reshape-education-curb-diversity-tighten-foreign-funding-rules/3547701)

**Real Summary:**  
White House allegedly targets advancing conservative agenda in education

**Generated Summary:**  
*Generated by: Module 6: Automated Keyword-Guided Summarizers (YAKE)*  
Executive orders aim to tighten financial pressure on universities and expand Trump’s influence on the education system as he pushes for diversity efforts in higher education

---

### Article: (Science-Technology)
[Read the article](https://www.aa.com.tr/en/science-technology/turkish-missile-producer-roketsan-inks-strategic-defense-deals-at-idef/3640692)

**Real Summary:**  
Roketsan attracts great interest at defense fair with its innovative products

**Generated Summary:**  
*Generated by: Module 6: Automated Keyword-Guided Summarizers (YAKE)*  
Roketsan attracted great interest at the fair with its innovative products and field-proven systems, becoming one of the most notable companies at the event

---

### Article: (Environment)
[Read the article](https://www.aa.com.tr/en/environment/scorching-heat-drought-a-new-normal-for-british-summers/3640907)

**Real Summary:**  
‘It’s hard to say this summer has been typical so far when we look at statistics,’ says national meteorological service scientist

**Generated Summary:**  
*Generated by: Module 2: Simple Prompters*  
With 3 heat waves already recorded and another warm surge on the way, summer 2025 could become a defining chapter in Britain’s climate history with its extreme temperatures, drought and wildfires

---

### Article: (Environment)
[Read the article](https://www.aa.com.tr/en/environment/interview-turkiye-can-lead-by-example-in-linking-growth-and-climate-action-un-climate-body-head/3641015)

**Real Summary:**  
‘Türkiye can lead by example, especially within the region, by showing how climate targets go hand-in-hand with economic growth,’ says Simon Stiell, head of the UN Framework Convention on Climate Change

**Generated Summary:**  
*Generated by: Module 5: Beam + Multiple Sampling Options*  
‘Türkiye has unique advantages – huge capacity for wind and solar, vibrant small and medium enterprises, and a young population. These advantages, combined with stronger targets, are a recipe for success,’ says Stiell  
‘Clear policies, climate ambition can support national growth, drive innovation, and create export opportunities,’ says Stiell during his 2-day visit to Türkiye

---
